Langgraph 기본 tool 사용법

In [1]:
from langchain_core.messages import AIMessage
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
import os
from dotenv import load_dotenv

load_dotenv()

@tool
def get_weather(location: str):
    """Call to get the weather"""
    if location in ["서울", "인천"]:
        return "It's 60 degrees and foggy."
    else:
        return "It's 90 degrees and sunny."

@tool
def get_coolest_cities():
    """Get a list of coolest cities"""
    return "서울, 고성"

tools = [get_weather, get_coolest_cities]
tool_node = ToolNode(tools)

from langchain_openai import ChatOpenAI

model_with_tools = ChatOpenAI(
    model="gpt-4o", temperature=0, api_key=os.getenv("OPENAI_API_KEY")
).bind_tools(tools)

model_with_tools.invoke("한국에서 가장 추운 도시는?").tool_calls

[{'name': 'get_coolest_cities',
  'args': {},
  'id': 'call_N194L8KhPotM89RqsZ7pkcJq',
  'type': 'tool_call'}]

In [2]:
model_with_tools.invoke("사과는 영어로 뭐야?").tool_calls

[]

conditional_edges와 tool_calls를 이용해서 도구 호출하는 방법

In [3]:
from typing import Annotated, Literal, TypedDict
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langgraph.graph import END, START, StateGraph, MessagesState

def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END


def call_model(state: MessagesState):
    messages = state['messages']
    response = model_with_tools.invoke(messages)
    return {"messages": [response]}


workflow = StateGraph(MessagesState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
)

workflow.add_edge("tools", 'agent')

app = workflow.compile()

In [4]:
final_state = app.invoke(
    {"messages": [HumanMessage(content="서울의 날씨는 어때?")]}
)
final_state["messages"][-1].content

'서울의 날씨는 현재 60도이며 안개가 끼어 있습니다.'

In [6]:
final_state = app.invoke(
    {"messages": [HumanMessage(content="서울의 날씨는 어때?")]}
)
final_state["messages"]

[HumanMessage(content='서울의 날씨는 어때?', additional_kwargs={}, response_metadata={}, id='31fdb18d-c5a4-4b91-b833-ab0ea1e19864'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xr0irkLhMu2uBfGt8kQ9tw3o', 'function': {'arguments': '{"location":"서울"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 70, 'total_tokens': 84, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_45cf54deae', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-003db776-1312-40c7-8ddf-44a7e03fcd7f-0', tool_calls=[{'name': 'get_weather', 'args': {'location': '서울'}, 'id': 'call_xr0irkLhMu2uBfGt8kQ9tw3o', 'type': 'tool_call'}], usage_metadata={'input_tokens': 70, 'output_tokens': 14, 'tot